In [ ]:
#Import ibm_db
import ibm_db

Connecting to dashDB or DB2 database requires the following information:<br>
Driver Name<br>
Database name<br>
Host DNS name or IP address<br>
Host port<br>
Connection protocol<br>
User ID (or username)<br>
User Password<br>

All these details you can get them once you added instance of DB2 service to your catalog at ibm cloud site ( You need to have account) and create new credentials

In [2]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "dashdb-txn-sbox-yp-lon02-07.services.eu-gb.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "bts74036"        # e.g. "abc12345"
dsn_pwd = "1mvxsg8czxn0f-l6"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB" 
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

In [3]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-lon02-07.services.eu-gb.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=bts74036;PWD=1mvxsg8czxn0f-l6;


In [4]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  bts74036 on host:  dashdb-txn-sbox-yp-lon02-07.services.eu-gb.bluemix.net


In [5]:
# Let us create table called 'INSTRUCTOR" with few rows and columns 

# first let us make sure there is no similar table hold the same name by dropping any exsisting table called "Instructor" ( make sure the exsisting table has no valuable information)

dropQuery = "drop table Instructor"

#Now execute the drop statment
dropStat = ibm_db.exec_immediate(conn,dropQuery)


In [6]:
# creating table query:
createQuery = 'create table Instructor(id Integer Primary Key Not Null,Fname varchar(50),Lname varchar(50),City varchar(50), ccode char(2))'
createStat = ibm_db.exec_immediate(conn,createQuery)

In [12]:
# Inser some values into Instructor table:

insertQuery = "insert into Instructor values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"
InsertStat = ibm_db.exec_immediate(conn,insertQuery)

In [53]:
# Let us run Select Query to check if the values have been inserted priperly:

selectQuery = "select * from instructor"
selectStat = ibm_db.exec_immediate(conn,selectQuery)

# Priniting the values for table rows:
while ibm_db.fetch_row(selectStat) != False:
    print("ID=", ibm_db.result(selectStat, 0), "First Name=", ibm_db.result(selectStat, 1), "Last Name=", ibm_db.result(selectStat,2), "City=", ibm_db.result(selectStat,3), "City Code=", ibm_db.result(selectStat,4))

ID= 1 First Name= Rav Last Name= Ahuja City= Vancouver City Code= CA
ID= 2 First Name= Raul Last Name= Chong City= Markham City Code= CA
ID= 3 First Name= Hima Last Name= Vasudevan City= Chicago City Code= US


In [58]:
# let us change the Rav city to Vancouver

updateQuery = "update Instructor set city='Vancouver' where Fname='Rav'"
updateStat = ibm_db.exec_immediate(conn,updateQuery)

# running the Select Query again to fetch the data from database:
selectQuery = "select * from instructor"
selectStat = ibm_db.exec_immediate(conn,selectQuery)
ibm_db.fetch_both(selectStat)
ibm_db.result(selectStat,3)

'Vancouver'

In [59]:
# let us retrieve Instructor table data into DataFrame using Panda library:

import pandas
import ibm_db_dbi

In [61]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [64]:
# Fitch data from datbase
selectQuery_P = "select * from Instructor"

# insert fitched data into DataFrame:
pdf = pandas.read_sql(selectQuery_P, pconn)

In [75]:
# display some data from DataFrame:

pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Vancouver,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [76]:
pdf.shape

(3, 5)

In [77]:
pdf.size

15

In [79]:
pdf[0:1]

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Vancouver,CA


In [82]:
pdf.CITY[1]

'Markham'

In [83]:
pdf.iloc[1,2]

'Chong'

In [84]:
#Dont forgot to close the Database connection:

ibm_db.close(conn)


True